# ENERJISA OPTIMIZATION CASE STUDY

In [6]:
import pandas as pd
import xpress as xp
import requests
import json
import plotly.graph_objs as go
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

In [7]:
url_price = 'https://seffaflik.epias.com.tr/transparency/service/market/day-ahead-mcp'

#Read the data with API
def read_data(url):
    
    headers = {
            'accept': 'application/json', }

        # Date filter
    params = {
            'startDate': '2021-01-01',
            'endDate': '2023-12-31'}

    response_price = requests.get(url_price, headers=headers, params=params)
    
        # Check if the request was successful
    if response_price.status_code == 200:

        #Parse
        data_price = response_price.json()

        # Extract the data from response
        records_price = data_price.get('body', {}).get('dayAheadMCPList', [])

        #Make it a df
        df_price = pd.DataFrame(records_price)

        #Convert date column to datetime
        if 'date' in df_price.columns:

            df_price['Tarih'] = pd.to_datetime(df_price['date']).dt.date
            df_price['Tarih'] = pd.to_datetime(df_price['Tarih'])

            df_price['Saat'] = pd.to_datetime(df_price['date']).dt.time

        return df_price

    else:
        print(f"Failed to retrieve data: {response.status_code}")

In [8]:
df_price = read_data(url_price)

In [9]:
#Filter the Date and some column type arrangements

df_price = df_price[(df_price["Tarih"]>='2023-12-25') & (df_price["Tarih"]<='2023-12-31')]

df_price = df_price.set_index("date")

df_price = df_price.rename(columns = {"price":"PTF (TL/MWh)", "priceUsd":"PTF (USD/MWh)", "priceEur":"PTF (EUR/MWh)"})

In [10]:
df_price

,PTF (TL/MWh),PTF (USD/MWh),PTF (EUR/MWh),Tarih,Saat
date,,,,,
2023-12-25T00:00:00.000+0300,1349.99,46.34,42.09,2023-12-25,00:00:00
2023-12-25T01:00:00.000+0300,1349.98,46.34,42.09,2023-12-25,01:00:00
2023-12-25T02:00:00.000+0300,1200.00,41.19,37.42,2023-12-25,02:00:00
2023-12-25T03:00:00.000+0300,979.99,33.64,30.56,2023-12-25,03:00:00
2023-12-25T04:00:00.000+0300,1248.99,42.87,38.94,2023-12-25,04:00:00
...,...,...,...,...,...
2023-12-31T19:00:00.000+0300,2499.67,84.91,76.74,2023-12-31,19:00:00
2023-12-31T20:00:00.000+0300,2472.34,83.98,75.90,2023-12-31,20:00:00
2023-12-31T21:00:00.000+0300,2472.33,83.98,75.90,2023-12-31,21:00:00


In [11]:
prices = df_price['PTF (TL/MWh)'].tolist()

# Power plant data
power_plants = [
    {"name": "A", "source": "Hydro", "capacity": 200, "cost": 1.6, "min_gen": 50},
    {"name": "B", "source": "Import Coal", "capacity": 500, "cost": 1.3, "min_gen": 200},
    {"name": "C", "source": "Natural Gas", "capacity": 900, "cost": 2.0, "min_gen": 300},
    {"name": "D", "source": "Natural Gas", "capacity": 600, "cost": 2.1, "min_gen": 200},
    {"name": "E", "source": "Hydro", "capacity": 400, "cost": 1.7, "min_gen": 50},
]

# Number of hours in the week 24 x 7 = 168
start_date = pd.Timestamp('2023-12-25 00:00:00')
end_date = pd.Timestamp('2023-12-31 23:00:00')
hours = pd.date_range(start=start_date, end=end_date, freq='H')

# Create the model
model = xp.problem()

# Decision Variables
generation = {(plant["name"], h): xp.var(vartype=xp.continuous, lb=0) for plant in power_plants for h in range(len(hours))}
is_on = {(plant["name"], h): xp.var(vartype=xp.binary) for plant in power_plants for h in range(len(hours))}

model.addVariable(generation, is_on)

# Objective function: Maximize asset value
asset_value = xp.Sum((prices[h] - plant["cost"]) * generation[plant["name"], h]
                     for plant in power_plants for h in range(len(hours)))
model.setObjective(asset_value, sense=xp.maximize)

# Constraints

# Demand should be met at each hour
for h in range(len(hours)):
    model.addConstraint(xp.Sum(generation[plant["name"], h] for plant in power_plants) == 1500)

# Generation constraints for each plant
for plant in power_plants:
    # power plant should be generating electricity more than its minimum generation requirement. 
    #A power plant can generate electricity maximum at its capacity.
    
    for h in range(len(hours)):
        model.addConstraint(generation[plant["name"], h] >= plant["min_gen"] * is_on[plant["name"], h])
        model.addConstraint(generation[plant["name"], h] <= plant["capacity"] * is_on[plant["name"], h])

    # Hydro constraint: maximum 80 hours per week
    if plant["source"] == "Hydro":
        model.addConstraint(xp.Sum(is_on[plant["name"], h] for h in range(len(hours))) <= 80)

    #  Coal constraint: minimum 12 consecutive hours
    if plant["source"] == "Import Coal":
        for h in range(len(hours) - 12 + 1):
            model.addConstraint(xp.Sum(is_on[plant["name"], h + i] for i in range(12)) >= 12)

# Model solve
model.solve()

#Generate the schedule
generation_schedule = {plant["name"]: [model.getSolution(generation[plant["name"], h]) for h in range(len(hours))] for plant in power_plants}

generation_schedule.items()


FICO Xpress v9.4.0, Community, solve started 0:47:28, May 21, 2024
Heap usage: 1469KB (peak 1469KB, 213KB system)
Maximizing MILP noname using up to 8 threads and up to 15GB memory, with these control settings:
OUTPUTLOG = 1
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = -1
Original problem has:
      2007 rows         1680 cols         6420 elements       840 entities
Presolved problem has:
       676 rows         1008 cols         2684 elements       672 entities
Presolve finished in 0 seconds
Heap usage: 1804KB (peak 3092KB, 213KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  9.00e+02] / [ 1.25e-01,  1.25e+01]
  RHS and bounds [min,max] : [ 1.00e+00,  1.50e+03] / [ 1.00e+00,  1.13e+02]
  Objective      [min,max] : [ 9.78e+02,  2.70e+03] / [ 3.20e+00,  6.40e+00]
Autoscaling applied Curtis-Reid scaling

Will try to keep branch and bound tree memory usage below 8.7GB
Starting concurrent solve w

C:\Users\furka\AppData\Local\Temp\ipykernel_7516\1612325668.py:18: LicenseWarning: Using the Community license in this session. If you have a full Xpress license, pass the full path to your license file to xpress.init(). If you want to use the FICO Community license and no longer want to see this message, use the following code before using the xpress module:
  xpress.init('C:/Users/furka/anaconda3/Lib/site-packages/xpress/license/community-xpauth.xpr')

  model = xp.problem()


STOPPING - MIPRELSTOP target reached (MIPRELSTOP=0.0001  gap=1.95915e-06).
 *** Search completed ***
Uncrunching matrix
Final MIP objective                   : 5.206333900000000e+08
Final MIP bound                       : 5.206344100000000e+08
  Solution time / primaldual integral :      0.08s/ 92.471054%
  Number of solutions found / nodes   :         1 /         0
  Max primal violation      (abs/rel) :       0.0 /       0.0
  Max integer violation     (abs    ) :       0.0


dict_items([('A', [200.0, 200.0, 0.0, 200.0, 0.0, 200.0, 0.0, 200.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 200.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 200.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 200.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 200.0, 200.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 0.0, 0.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 0.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 0.0, 0.0, 0.0, 0

In [12]:
# Create a DataFrame for easier plotting
generation_df = pd.DataFrame(generation_schedule, index=hours)

# Initialize the Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Power Plant Generation Schedule"),
    dcc.Graph(id='generation-graph'),
    dcc.Dropdown(
        id='plant-dropdown',
        options=[{'label': plant['name'], 'value': plant['name']} for plant in power_plants],
        value=[plant['name'] for plant in power_plants],
        multi=True
    )
])

@app.callback(
    Output('generation-graph', 'figure'),
    [Input('plant-dropdown', 'value')]
)
def update_graph(selected_plants):
    fig = go.Figure()
    for plant in selected_plants:
        fig.add_trace(go.Scatter(
            x=generation_df.index,
            y=generation_df[plant],
            mode='lines',
            name=f'Power Plant {plant}'
        ))

    fig.update_layout(
        title="Hourly Generation Schedule for Power Plants",
        xaxis_title="Date and Time",
        yaxis_title="Generation (MW)",
        legend_title="Power Plant",
        xaxis=dict(tickformat='%Y-%m-%d %H:%M')
    )
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)